# モデル開発
- LightGBMで回帰モデルを作成

In [46]:
import pandas as pd
import numpy as np
import pickle
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import pandas as pd

# RMSEを計算する関数
def rmse(validation, target):
    return np.sqrt(mean_squared_error(validation, target))

# 学習データと検証データの読み込み
df_train = pd.read_csv("../data/Walmert_train.csv")
df_valid = pd.read_csv("../data/Walmert_valid.csv")

# Date列からMonth列とDay列を追加し、Date列を削除
df_train['Month'] = pd.to_datetime(df_train['Date']).dt.month
df_train['Day'] = pd.to_datetime(df_train['Date']).dt.day
df_train = df_train.drop(columns='Date')
df_valid['Month'] = pd.to_datetime(df_valid['Date']).dt.month
df_valid['Day'] = pd.to_datetime(df_valid['Date']).dt.day
df_valid = df_valid.drop(columns='Date')

# ターゲット変数となる列名を指定
col_target = "Weekly_Sales"

# 学習データと検証データを、特徴量とターゲット変数に分割
X_train = df_train.drop(columns=col_target)
y_train = df_train[col_target].to_numpy().ravel()
X_valid = df_valid.drop(columns=col_target)
y_valid = df_valid[col_target].to_numpy().ravel()

# LightGBMのデータセットに変換
train_data = lgb.Dataset(X_train, label=y_train)

# ハイパーパラメータの設定
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05
}

# モデルの学習
model = lgb.train(params, train_data, num_boost_round=100)

# モデルの保存
pickle.dump(model, open("./model.pkl", "wb"))

# 検証データでRMSEを算出
preds = model.predict(X_valid)
print(rmse(df_valid[col_target], preds))


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000191 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1090
[LightGBM] [Info] Number of data points in the train set: 4095, number of used features: 8
[LightGBM] [Info] Start training from score 1035127.148176
271869.75550026493


# AzureMLワークスペース接続
- 接続のためのクレデンシャル取得

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

- AzureMLワークスペースへ接続するためのハンドルを取得

In [2]:
ml_client = MLClient(
    credential=credential,
    subscription_id="27a05524-7a59-453b-88d7-df3ebaa2bcc1",
    resource_group_name="azureml-book",
    workspace_name="azureml-book",
)

# データアセット登録
- 学習データをアセット登録

In [31]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# 登録するデータアセットのバージョン指定(例: "1")
VERSION = "1"

# 学習データのパス指定
# local: './<path>/<file>' (対象データは自動的にデフォルトのデータストアへアップロードされる)
# blob:  'wasbs://<container_name>@<account_name>.blob.core.windows.net/<path>/<file>'
# ADLS gen2: 'abfss://<file_system>@<account_name>.dfs.core.windows.net/<path>/<file>'
# Datastore: 'azureml://datastores/<data_store_name>/paths/<path>/<file>'
path = "../data/Walmert_train.csv"

# 学習データのアセット定義
my_data = Data(
    path=path,
    type=AssetTypes.URI_FILE,
    description="ウォルマートの売上履歴　学習データセット",
    name="Walmart_store_sales_train",
    version=VERSION,
)

# 学習データのアセット作成
ml_client.data.create_or_update(my_data)

Data({'path': 'azureml://subscriptions/27a05524-7a59-453b-88d7-df3ebaa2bcc1/resourcegroups/azureml-book/workspaces/azureml-book/datastores/workspaceblobstore/paths/LocalUpload/e0f17b75a81915454376a037b34944c6/Walmert_train.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'Walmart_store_sales_train', 'description': 'ウォルマートの売上履歴の学習データセット', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/27a05524-7a59-453b-88d7-df3ebaa2bcc1/resourceGroups/azureml-book/providers/Microsoft.MachineLearningServices/workspaces/azureml-book/data/Walmart_store_sales_train/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/azureml-book/code/Users/yutatatewaki/azureml-book/ch5', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fb1756c1030>, 'serialize': <msre

- 検証データをアセット登録

In [35]:
# 検証データのパス指定
path = "../data/Walmert_valid.csv"

# 検証データのアセット定義
my_data = Data(
    path=path,
    type=AssetTypes.URI_FILE,
    description="ウォルマートの売上履歴　検証データセット",
    name="Walmart_store_sales_valid",
    version=VERSION,
)

# 検証データのアセット作成
ml_client.data.create_or_update(my_data)

Uploading Walmert_valid.csv (< 1 MB): 100%|██████████| 65.2k/65.2k [00:00<00:00, 5.25MB/s]




Data({'path': 'azureml://subscriptions/27a05524-7a59-453b-88d7-df3ebaa2bcc1/resourcegroups/azureml-book/workspaces/azureml-book/datastores/workspaceblobstore/paths/LocalUpload/00ae3501ebdca92d9b5f197a706c47e1/Walmert_valid.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'Walmart_store_sales_valid', 'description': 'ウォルマートの売上履歴\u3000検証データセット', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/27a05524-7a59-453b-88d7-df3ebaa2bcc1/resourceGroups/azureml-book/providers/Microsoft.MachineLearningServices/workspaces/azureml-book/data/Walmart_store_sales_valid/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/azureml-book/code/Users/yutatatewaki/azureml-book/ch5', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fb1756c0790>, 'serialize': 

# 環境の作成

In [12]:
import os
from azure.ai.ml.entities import Environment

custom_env_name = "walmart-store-sales-env"
env_dir = "./env"

custom_job_env = Environment(
    name=custom_env_name,
    description="ウォルマート売上予測モデルの学習ジョブ用の環境",
    tags={"lightgbm": "4.3.0"},
    conda_file=os.path.join(env_dir, "environment.yml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
custom_job_env = ml_client.environments.create_or_update(custom_job_env)

print(
    f"{custom_job_env.name} 環境をAzureMLワークスペースへ登録しました。環境バージョンは {custom_job_env.version} です。"
)

walmart-store-sales-env 環境をAzureMLワークスペースへ登録しました。環境バージョンは 2 です。


In [12]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import mlflow
import mlflow.lightgbm

# ロギング開始
mlflow.start_run()

# 自動ロギング有効化
mlflow.lightgbm.autolog()

# 学習データと検証データの読み込み
data_asset = ml_client.data.get(name="Walmart_store_sales_train", version=1)
df_train = pd.read_csv(data_asset.path)
data_asset = ml_client.data.get(name="Walmart_store_sales_valid", version=1)
df_valid = pd.read_csv(data_asset.path)

# Date列からMonth列とDay列を追加し、Date列を削除
df_train['Month'] = pd.to_datetime(df_train['Date']).dt.month
df_train['Day'] = pd.to_datetime(df_train['Date']).dt.day
df_train = df_train.drop(columns='Date')
df_valid['Month'] = pd.to_datetime(df_valid['Date']).dt.month
df_valid['Day'] = pd.to_datetime(df_valid['Date']).dt.day
df_valid = df_valid.drop(columns='Date')

# ターゲット変数となる列名を指定
col_target = "Weekly_Sales"

# 学習データと検証データを、特徴量とターゲット変数に分割
X_train = df_train.drop(columns=col_target)
y_train = df_train[col_target].to_numpy().ravel()
X_valid = df_valid.drop(columns=col_target)
y_valid = df_valid[col_target].to_numpy().ravel()

# LightGBMのデータセットに変換
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

# ハイパーパラメータの設定
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05
}

# モデルの学習
model = lgb.train(params=params, train_set=train_data,
                    num_boost_round=100, valid_sets=valid_data)

##########################
#<モデル登録>
##########################
# 学習済みモデルをAzureMLワークスペースへ登録
print("MLflowでモデル登録")
mlflow.lightgbm.log_model(
    lgb_model=model,
    registered_model_name='Walmart_store_sales_model',
    artifact_path='Walmart_store_sales_model',
)

###########################
#</モデル登録>
###########################

# ロギング停止
mlflow.end_run()


AttributeError: partially initialized module 'pandas' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

In [23]:
%%writefile ./src/main.py
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
import numpy as np
import lightgbm as lgb
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential


def main():
    """メイン関数"""

    # パラメータ
    parser = argparse.ArgumentParser()
    parser.add_argument("--num_leaves", type=int, default=31, help="学習率")
    parser.add_argument("--learning_rate", type=float, default=0.05, help="1本の木の最大葉枚数")
    parser.add_argument("--registered_model_name", type=str, help="登録するモデル名")
    parser.add_argument("--train_data_path", type=str, help="学習データアセットパス")
    parser.add_argument("--valid_data_path", type=str, help="検証データアセットパス")

    args = parser.parse_args()

    # AzureMLワークスペースへの接続
    credential = DefaultAzureCredential(exclude_workload_identity_credential=True)
    ml_client = MLClient(
        credential=credential,
        subscription_id="<SUBSCRIPTION_ID>",
        resource_group_name="<RESOURCE_GROUP>",
        workspace_name="<AML_WORKSPACE_NAME>",
    )

    # ロギング開始
    mlflow.start_run()

    # 自動ロギング有効化
    mlflow.lightgbm.autolog()

    ###################
    #<データ準備>
    ###################

    # 学習データと検証データの読み込み
    df_train = pd.read_csv(args.train_data_path)
    df_valid = pd.read_csv(args.valid_data_path)

    # Date列からMonth列とDay列を追加し、Date列を削除
    df_train['Month'] = pd.to_datetime(df_train['Date']).dt.month
    df_train['Day'] = pd.to_datetime(df_train['Date']).dt.day
    df_train = df_train.drop(columns='Date')
    df_valid['Month'] = pd.to_datetime(df_valid['Date']).dt.month
    df_valid['Day'] = pd.to_datetime(df_valid['Date']).dt.day
    df_valid = df_valid.drop(columns='Date')
    
    # ターゲット変数となる列名を指定
    col_target = "Weekly_Sales"

    # 学習データと検証データを、特徴量とターゲット変数に分割
    X_train = df_train.drop(columns=col_target)
    y_train = df_train[col_target].to_numpy().ravel()
    X_valid = df_valid.drop(columns=col_target)
    y_valid = df_valid[col_target].to_numpy().ravel()

    # LightGBMのデータセットに変換
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)

    ####################
    #</データ準備>
    ####################

    ##################
    #<学習>
    ##################
    # ハイパーパラメータの設定
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': args.num_leaves,
        'learning_rate': args.learning_rate
    }

    # モデルの学習
    model = lgb.train(params=params, train_set=train_data,
                        num_boost_round=100, valid_sets=valid_data)


    ###################
    #</学習>
    ###################

    ##########################
    #<モデル登録>
    ##########################
    # 学習済みモデルをAzureMLワークスペースへ登録
    mlflow.lightgbm.log_model(
        lgb_model=model,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name
    )

    ###########################
    #</モデル登録>
    ###########################

    # ロギング停止
    mlflow.end_run()

if __name__ == "__main__":
    main()

Overwriting ./src/main.py


In [26]:
from azure.ai.ml import command
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

train_data_name = ml_client.data.get(name="Walmart_store_sales_train", version="1")
valid_data_name = ml_client.data.get(name="Walmart_store_sales_valid", version="1")

# 学習スクリプト引数設定
inputs = {
    # ジョブの入力として学習データ指定
    "train_data_path": Input(
            type=AssetTypes.URI_FILE,
            path=train_data_name.id
    ),
    # ジョブの入力として検証データ指定
    "valid_data_path": Input(
            type=AssetTypes.URI_FILE,
            path=valid_data_name.id
    ),
    # 1本の木の最大葉枚数
    "num_leaves" : 30,
    # 学習率
    "learning_rate" : 0.04,
    # 登録するモデル名
    "registered_model_name" : "Walmart_store_sales_model"
}

# 学習ジョブの構成
job = command(
    # 学習スクリプト引数
    inputs=inputs,
    # 学習スクリプトの格納場所
    code="./src/",
    # 学習スクリプトの実行コマンド
    #command="python main.py --num_leaves ${{inputs.num_leaves}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}} --train_data_name ${{inputs.train_data_name}} --train_data_version ${{inputs.train_data_version}} --valid_data_name ${{inputs.valid_data_name}} --valid_data_version ${{inputs.valid_data_version}}",
    command="python main.py --num_leaves ${{inputs.num_leaves}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}} --train_data_path ${{inputs.train_data_path}} --valid_data_path ${{inputs.valid_data_path}}",
    # 環境 ( @latest で最新版を指定。 :バージョン数 でバージョン指定も可)
    environment="walmart-store-sales-env@latest",
    # 実験名
    experiment_name="train_walmart_store_sales_prediction",
    # ジョブの表示名
    display_name="walmart_store_sales_prediction"
)

In [27]:
job = ml_client.create_or_update(job)
# ジョブ実行が完了するまで待機
ml_client.jobs.stream(job.name)

RunId: witty_chaconia_g10yqj5gbm
Web View: https://ml.azure.com/runs/witty_chaconia_g10yqj5gbm?wsid=/subscriptions/27a05524-7a59-453b-88d7-df3ebaa2bcc1/resourcegroups/azureml-book/workspaces/azureml-book

Execution Summary
RunId: witty_chaconia_g10yqj5gbm
Web View: https://ml.azure.com/runs/witty_chaconia_g10yqj5gbm?wsid=/subscriptions/27a05524-7a59-453b-88d7-df3ebaa2bcc1/resourcegroups/azureml-book/workspaces/azureml-book

